In [1]:
from pandas import read_csv
from matplotlib import pyplot
import sys
import numpy as np
import pandas as pd


import random

random.seed(10)

N=9
nowcast=1

y_input_size=1
z_input_size=1

file="univer_segmented.csv"

series=read_csv(file, sep=",", header=0,parse_dates=True)
series=series.values
series


array([[0, '2019-06-09 00:10:00+00:00', 0.0, 17.362500000000004, 0.0],
       [1, '2019-06-09 00:20:00+00:00', 0.0, 17.2375, 0.0],
       [2, '2019-06-09 00:30:00+00:00', 0.0, 17.170588235294115, 0.0],
       ...,
       [24028, '2019-11-22 23:40:00+00:00', 0.0, 9.08125, 0.0],
       [24029, '2019-11-22 23:50:00+00:00', 0.0, 9.08, 0.25],
       [24030, '2019-11-23 00:00:00+00:00', 0.0, 9.183333333333334,
        1.8458333333333328]], dtype=object)

In [2]:

X1=sensors=np.array(series[:,2])
X2=sensors=np.array(series[:,3])
Y0=sensors=np.array(series[:,4])

print(X1[0])
print(X2[0])
print(Y0[0])

0.0
17.362500000000004
0.0


In [3]:
import tensorflow as tf 

X1n=tf.keras.utils.normalize(    X1, axis=-1, order=2)
X1n=np.array(X1n[0])
#X1n=X1


print(X1n)
print(X1n.shape)

X2n=tf.keras.utils.normalize(X2, axis=-1, order=2)
X2n=np.array(X2n[0])
#X2n=X2
print(X2n)
print(X2n.shape)

X0=np.zeros((len(X1n),2))

X0[:,0]=X1n
X0[:,1]=X2n

Y=np.zeros((len(Y0),1))
Y[:,0]=Y0

print(X0)
print(X0.shape)
print(Y.shape)
print(Y)

[0.0 0.0 0.0 ... 0.0 0.0 0.0]
(24031,)
[0.005448714089054488 0.005409486485821554 0.00538848818056157 ...
 0.0028498853748726316 0.002849493098840302 0.0028819212508461936]
(24031,)
[[0.         0.00544871]
 [0.         0.00540949]
 [0.         0.00538849]
 ...
 [0.         0.00284989]
 [0.         0.00284949]
 [0.         0.00288192]]
(24031, 2)
(24031, 1)
[[0.        ]
 [0.        ]
 [0.        ]
 ...
 [0.        ]
 [0.25      ]
 [1.84583333]]


In [4]:
X=[]
for i in range(N,len(X0)):
    X.append(X0[i-N:i])
X=np.array(X)    
print(X.shape)

Y=Y[N-nowcast:len(Y)-nowcast]
print(Y.shape)



from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
print(Y[40])
scaler.fit(Y)

Y = scaler.transform(Y)
print(Y[40])
print("max:",scaler.data_max_)
print("min:",scaler.data_min_)


(24022, 9, 2)
(24022, 1)
[10456.769375]
[0.36726104]
max: [28472.30769231]
min: [0.]


In [5]:
print(X[0])
print(Y[0])

[[0.         0.00544871]
 [0.         0.00540949]
 [0.         0.00538849]
 [0.         0.00547189]
 [0.         0.00538595]
 [0.         0.00530161]
 [0.         0.00520943]
 [0.         0.00517804]
 [0.         0.00518432]]
[0.]


In [6]:
from sklearn.model_selection import train_test_split
x_train , x_val , y_train , y_val = train_test_split(X , Y ,test_size=0.2, random_state=0)

In [7]:
from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Input, Conv1D, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D, concatenate
from tensorflow.keras.models import Model, Sequential

from keras import backend as K


mae = tf.keras.losses.MeanAbsoluteError()


def make_generator():
    x = Input(shape=(N,2))    
    x_output = Conv1D(filters=16, kernel_size=2, activation='relu')(x)
    x_output = Conv1D(filters=32, kernel_size=3, activation='relu')(x_output)
    x_output = Conv1D(filters=64, kernel_size=3, activation='relu')(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32, return_sequences=True)(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32)(x_output)
    latent = Flatten()(x_output)

    noise = Input(shape=(z_input_size,))
    noise_output = Dense(64, activation='relu')(noise)

    flatten = concatenate([latent, noise_output])

    
    concat = Dense(256)(flatten)
    concat = Dense(128)(concat)
    y_fake = Dense(1,activation="sigmoid")(concat)

    model = Model(inputs=[x,noise], outputs=y_fake)
    return model  






def make_discriminator():
    
    x = Input(shape=(N,2))    
    x_output = Conv1D(filters=16, kernel_size=2, activation='relu')(x)
    x_output = Conv1D(filters=32, kernel_size=3, activation='relu')(x_output)
    x_output = Conv1D(filters=64, kernel_size=3, activation='relu')(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32, return_sequences=True)(x_output)
    x_output = Dropout(0.25)(x_output)
    x_output = LSTM(32)(x_output)
    latent = Flatten()(x_output)

    label = Input(shape=(y_input_size,))
    label_output = Dense(512, activation='relu')(label)
    label_output = Dense(256, activation='relu')(label_output)

    concat = concatenate([latent, label_output])
    concat = Dense(256)(concat)
    concat = Dense(128)(concat)
    validity = Dense(1,activation="sigmoid")(concat)

    model = Model(inputs=[x, label], outputs=validity)
    model.compile(loss=['binary_crossentropy'],optimizer='adam')
    model.summary()
    return model  

x = Input(shape=(N,2))
noise = Input(shape=(z_input_size,))
generator = make_generator()
label_fake = generator([x,noise])


discriminator = make_discriminator()

validity = discriminator([x, label_fake])

discriminator.trainable = False

combined = Model([noise, x], [label_fake,validity])
combined.compile(loss=['binary_crossentropy'],optimizer='adam',metrics=['accuracy'])
#combined.compile(loss=loss_error_gan,optimizer='adam')
combined.summary()

#combined = Sequential()
#combined.add(generator)
#combined.add(lier)
#combined.add(discriminator)
#combined.compile(loss='binary_crossentropy', optimizer='adam')
    


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 9, 2)]       0                                            
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 8, 16)        80          input_5[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 6, 32)        1568        conv1d_3[0][0]                   
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 4, 64)        6208        conv1d_4[0][0]                   
____________________________________________________________________________________________

In [8]:
print(x_train.shape)
print(y_train.shape)



#history = generator.fit(x_train,y_train ,
#                    batch_size=64,
#            validation_data=(x_val,y_val),
#            epochs=40)

(19217, 9, 2)
(19217, 1)


In [9]:
#from sklearn.metrics import confusion_matrix
#yhat = generator.predict(x_val)
#yhat=scaler.inverse_transform(yhat).flatten()
#y_val0=scaler.inverse_transform(y_val).flatten()
#print(y_val)
#print(yhat)

#file = open('prediction.csv', 'w')
#file.write('ground truth\tprediction\n')
#for i in range(0,len(y_val)):
#    file.write(str(y_val0[i])+"\t"+str(yhat[i])+"\n")
#file.close()

#print("RMSE")
#error=tf.keras.metrics.mean_squared_error(
#    y_val0, yhat
#)
#print(np.sqrt(error))

In [10]:
def generate_true_samples(batch_size):
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    x, true_labels = x_train[idx], y_train[idx]
    return x,true_labels

def getNoise():
    return  np.random.normal(0, 1, (batch_size, z_input_size))



In [11]:
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

    
n_epochs=40
batch_size=64

valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))


file = open('metrics_cgan_class.csv', 'w')

for epoch in range(n_epochs):
    print("e:",epoch)
    for batch_idx in range(int(x_train.shape[0] // batch_size)):
        
        
        noise = getNoise()
        x,true_labels = generate_true_samples(batch_size)
        fake_labels = generator.predict([x,noise])

        discriminator.train_on_batch([x, true_labels], valid)
        discriminator.train_on_batch([x, fake_labels+0.1* tf.random.uniform(tf.shape(fake_labels))], fake)
        
        x,true_labels = generate_true_samples(batch_size)
        
        combined.train_on_batch([noise,x], [true_labels,valid])
        
        

    print(epoch, " of ", n_epochs)
    
    
    noise = np.random.normal(0, 1, (x_val.shape[0], z_input_size))
    yhat = generator.predict([x_val,noise])
   
    yhat0=scaler.inverse_transform(yhat).flatten()
    y_val0=scaler.inverse_transform(y_val).flatten()

    error0=tf.keras.metrics.mean_squared_error(
        y_val0, yhat0
    )
    print(np.sqrt(error0))    
    file.write(str(np.sqrt(error0))+'\t')
    
    error=tf.keras.metrics.mean_squared_error(
        y_val.flatten(), yhat.flatten()
    )
    print(np.sqrt(error))    
    file.write(str(np.sqrt(error))+'\t')
    
    pred_true=discriminator.predict([x_val, y_val])
    print("pred_true:",pred_true)
    print(pred_true[pred_true>0.5])
    acc_true=len(pred_true[pred_true>=0.5])/len(y_val)   
    file.write(str(acc_true)+'\t')


    pred_false=discriminator.predict([x_val, yhat])
    print("pred_false:",pred_false)
    acc_false=len(pred_false[pred_false<0.5])/len(y_val)   
    file.write(str(acc_false)+'\t')

    pred_false2=discriminator.predict([x_val, yhat+0.5* tf.random.uniform(tf.shape(yhat))])
    print("pred_false:",pred_false2)
    acc_false2=len(pred_false2[pred_false2<0.5])/len(y_val)   
    file.write(str(acc_false2)+'\n')

    print(str(np.sqrt(error))+'\t')

    print(str(acc_true)+'\t'+str(np.mean(pred_true)))
    print(str(acc_false)+'\t'+str(np.mean(pred_false)))
    print(str(acc_false2)+'\t'+str(np.mean(pred_false)))

    
file.close()        
        
    #noise = getNoise()
    #x,true_labels = generate_true_samples(batch_size)
    #fake_labels = generator.predict([x,noise])
    #print("true_labels:",true_labels)
    #print("fake:",fake_labels)
    
    #print("g_loss:",g_loss)
    #print("d_loss:",d_loss)

e: 0
0  of  40
2491.064
0.087490775
pred_true: [[0.9390223 ]
 [0.47793266]
 [0.19616109]
 ...
 [0.4869357 ]
 [0.4868406 ]
 [0.97659457]]
[0.9390223  0.97659457 0.97497785 ... 0.97580266 0.97659457 0.97659457]
pred_false: [[0.61971945]
 [0.4865714 ]
 [0.12902448]
 ...
 [0.48645133]
 [0.47897404]
 [0.8246533 ]]
pred_false: [[0.48778316]
 [0.47493103]
 [0.47850442]
 ...
 [0.4690289 ]
 [0.47361508]
 [0.47745094]]
0.087490775	
0.49157127991675337	0.67516077
0.8183142559833507	0.4092
0.9981269510926118	0.4092
e: 1


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import confusion_matrix

noise = np.random.normal(0, 1, (x_val.shape[0], z_input_size))
yhat = generator.predict([x_val,noise])
    
yhat=scaler.inverse_transform(yhat).flatten()
y_val0=scaler.inverse_transform(y_val).flatten()
print(y_val)
print(yhat)

file = open('prediction2.csv', 'w')
file.write('ground truth\tprediction\n')
for i in range(0,len(y_val)):
    file.write(str(y_val0[i])+"\t"+str(yhat[i])+"\n")
file.close()

print("RMSE")
error=tf.keras.metrics.mean_squared_error(
    y_val0, yhat
)
print(np.sqrt(error))